In [1]:
!/Users/katerinakuhne/opt/anaconda3/bin/python -m pip install kaggle_environments

In [60]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

In [61]:
%%writefile rock_agent.py

#0 - rock
#1 - paper
#2 - scissors

#Агент, который всегда выбирает "камень" (1)

def rock_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


In [62]:
%%writefile paper_agent.py

#Агент, который всегда выбирает "бумагу" (2)

def paper_agent(observation, configuration):
    return 1

Overwriting paper_agent.py


In [63]:
%%writefile scissors_agent.py

#Агент, который всегда выбирает "ножницы" (3)

def scissors_agent(observation, configuration):
    return 2

Overwriting scissors_agent.py


In [104]:
%%writefile random_agent.py
import random

#Агент, который всегда выбирает ход случайно (4)

def random_agent(observation, configuration):
    return random.randint(0, 2)

Overwriting random_agent.py


In [74]:
%%writefile rock_paper_agent.py
import random

#Агент, который случайно выбирает между "камнем" и "бумагой" (5)

def rock_paper_agent(observation, configuration):
    return random.randint(0, 1)

Overwriting rock_paper_agent.py


In [75]:
%%writefile paper_scissors_agent.py
import random

#Агент, который случайно выбирает между "бумагой" и "ножницами" (6)

def paper_scissors_agent(observation, configuration):
    return random.randint(1, 2)

Overwriting paper_scissors_agent.py


In [76]:
%%writefile mirror_agent.py
import random

#Агент, который копирует прошлый ход другого игрока (7)

def mirror_agent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

Overwriting mirror_agent.py


In [77]:
%%writefile contradictious_agent.py
import random

#Агент, который случайным образом выбирает ход, отличный от прошлого хода другого игрока (8)

def contradictious_agent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction+random.randint(1, 2))%3  
    else:
        return random.randrange(0, configuration.signs)

Overwriting contradictious_agent.py


In [78]:
%%writefile boring_agent.py
import random

#Агент, который повторяет свой прошлый выигрышный ход и меняет ход, если проиграл (9)  

def get_score(left_move, right_move):
    """
    Функция для вычисления исхода последней игры. 
    Возвращает 0 при ничьей, 1 при выигрыше и -1 при проигрыше.
    """
    if left_move == right_move:
        return 0  # ничья
    elif (left_move == 0 and right_move == 2) or \
         (left_move == 1 and right_move == 0) or \
         (left_move == 2 and right_move == 1):
        return 1  # победа левого игрока
    else:
        return -1  # проигрыш левого игрока

def boring_agent(observation, configuration):
    global last_counter_action
    if observation.step == 0:
        last_counter_action = random.randrange(0, configuration.signs)  # случайный первый ход
    elif get_score(last_counter_action, observation.lastOpponentAction) == 1:
        last_counter_action = last_counter_action  # если выиграл, повторяет ход
    else:
        last_counter_action = (last_counter_action+1)%3   # если проиграл, выбирает новый ход

    return last_counter_action    

Overwriting boring_agent.py


In [79]:
%%writefile great_gambits_agent.py
import random

#Агент, который каждые три хода рандомно выбирает один из восьми "Великих гамбитов" (10)

COMBINATIONS = [
    [0, 0, 0],  # Лавина (камень, камень, камень)
    [1, 1, 1],  # Клерк (бумага, бумага, бумага)
    [2, 2, 2],  # Ящик с инструментами (ножницы, ножницы, ножницы)
    [1, 2, 0],  # Крещендо (бумага, ножницы, камень)
    [0, 2, 1],  # Развязка (камень, ножницы, бумага)
    [0, 1, 1],  # Пачка денег (камень, бумага, бумага)
    [2, 1, 1],  # Бумажная кукла (ножницы, бумага, бумага)
    [1, 2, 1]   # Сэндвич (бумага, ножницы, бумага)
]

def great_gambits_agent(observation, configuration):
    global current_combination
    global move_index
    if observation.step == 0:
        current_combination = random.choice(COMBINATIONS)  #Выбирает первую комбинацию на первом ходу
        move_index = 0

    move = current_combination[move_index] #Выбирает текущий ход из комбинации
    move_index += 1

    #Когда прошло три хода, выбирает новую комбинацию
    if move_index == 3:
        current_combination = random.choice(COMBINATIONS)
        move_index = 0

    return move

Overwriting great_gambits_agent.py


In [91]:
%%writefile gambits_shifter_agent.py
import random

#Агент, который переключается на другой "Великий гамбит", только если проиграл в предыдущей серии из 3-ех ходов (11)

COMBINATIONS = [
    [0, 0, 0],  # Лавина (камень, камень, камень)
    [1, 1, 1],  # Клерк (бумага, бумага, бумага)
    [2, 2, 2],  # Ящик с инструментами (ножницы, ножницы, ножницы)
    [1, 2, 0],  # Крещендо (бумага, ножницы, камень)
    [0, 2, 1],  # Развязка (камень, ножницы, бумага)
    [0, 1, 1],  # Пачка денег (камень, бумага, бумага)
    [2, 1, 1],  # Бумажная кукла (ножницы, бумага, бумага)
    [1, 2, 1]   # Сэндвич (бумага, ножницы, бумага)
]

def get_score(left_move, right_move):
    """
    Функция для вычисления исхода последней игры. 
    Возвращает 0 при ничьей, 1 при выигрыше и -1 при проигрыше.
    """
    if left_move == right_move:
        return 0  # ничья
    elif (left_move == 0 and right_move == 2) or \
         (left_move == 1 and right_move == 0) or \
         (left_move == 2 and right_move == 1):
        return 1  # победа левого игрока
    else:
        return -1  # проигрыш левого игрока

def gambits_shifter_agent(observation, configuration):
    global current_combination, move_index, last_move, score_count
    
    if observation.step == 0:
        current_combination = random.choice(COMBINATIONS)  #Выбор комбинации на первом ходу
        move_index = 0
        score_count = 0
    
    move = current_combination[move_index]  #Выбор текущего хода
    move_index += 1

    if observation.step > 0:
        score_count += get_score(last_move, observation.lastOpponentAction) #Подсчёт счёта после каждого хода

    if move_index == 3:
        if score_count <= 1:  
            current_combination = random.choice(COMBINATIONS) #Смена комбинации при плохом результате
        move_index = 0
        score_count = 0

    last_move = move

    return move

Overwriting gambits_shifter_agent.py


In [81]:
%%writefile pattern_agent.py
import random

#Агент, который анализирует последовательности из 3 ходов противника в поисках повторяющихся паттернов, 
#и если паттерн найден, делает выигрышный контр-ход в ответ на ожидаемый ход противника (12)

def get_counter_move(opponent_move):
    """
    Принимает ход противника и возвращает выигрышный ход
    """
    return (opponent_move + 1) % 3  

def find_pattern(history, n=2):
    """
    Ищет повторяющиеся последовательности длиной n в истории ходов противника.
    Возвращает предположительный следующий ход противника, если паттерн найден.
    """
    if len(history) < n:
        return None

    recent_pattern = tuple(history[-n:])  #Последние n ходов противника

    for i in range(len(history) - n):
        if tuple(history[i:i+n]) == recent_pattern:
            if i + n < len(history):
                return history[i + n]  

    return None

def pattern_agent(observation, configuration):
    global opponent_history

    if observation.step == 0:
        opponent_history = []

    if observation.step > 0:
        opponent_history.append(observation.lastOpponentAction)

    predicted_move = find_pattern(opponent_history, n=4) #Ищем паттерн длиной 4

    if predicted_move is not None:
        return get_counter_move(predicted_move) 
    else:
        return random.randint(0, 2)

Overwriting pattern_agent.py


In [100]:
%%writefile q_learning_agent.py
import random

#Агент, использующий Q-learning алгоритм с политикой Epsilon-Greedy для выбора хода (13)

def get_score(left_move, right_move):
    """
    Функция для вычисления исхода последней игры. 
    Возвращает 0 при ничьей, 1 при выигрыше и -1 при проигрыше.
    """
    if left_move == right_move:
        return 0  # ничья
    elif (left_move == 0 and right_move == 2) or \
         (left_move == 1 and right_move == 0) or \
         (left_move == 2 and right_move == 1):
        return 1  # победа левого игрока
    else:
        return -1  # проигрыш левого игрока

q_values = [0, 0, 0]  #таблица выигрышей для камня, ножниц, бумаги
alpha = 0.1  #коэффициент обучения
gamma = 0.9  #коэффициент дисконтирования, чтобы уменьшить "значимость" награды в будущих играх
epsilon = 0.1  #вероятность выбора новой стратегии (случайного хода)
last_action = None

def q_learning_agent(observation, configuration):
    global q_values
    global last_action

    if observation.step == 0:
        last_action = random.randint(0, 2) #наш агент деалет первый ход случайным образом
        return last_action 
    
    opponent_last_action = observation.lastOpponentAction #последний ход оппонента
    reward = get_score(last_action, opponent_last_action) #вычисляем исход последней игры и исходя из этого назначаем награду
    
    #обновляем Q-значение для предыдущего хода
    q_values[last_action] = q_values[last_action] + alpha * (reward + gamma * max(q_values) - q_values[last_action])

    #решаем каким будет следующий ход
    if random.random() < epsilon:
        action = random.randint(0, 2)  #если выпавшее рандомное число меньше 0.1, делаем случайный ход
    else:
        action = q_values.index(max(q_values))  #иначе выбираем ход с максимальной оценкой

    last_action = action  
    return action

Overwriting q_learning_agent.py


In [110]:
#Турнир между агентами
agents_py = ['rock_agent.py', 'paper_agent.py', 'scissors_agent.py',
          'random_agent.py', 'rock_paper_agent.py', 
          'paper_scissors_agent.py', 'mirror_agent.py',
          'contradictious_agent.py', 'boring_agent.py',
          'great_gambits_agent.py', 'gambits_shifter_agent.py',
          'pattern_agent.py', 'q_learning_agent.py']

n_agents = len(agents_py)

#Инициализируем матрицу результатов
scores = np.zeros((n_agents, n_agents))

#Играем каждый агент против каждого
for i in range(n_agents):
    for j in range(i + 1, n_agents):
        result = evaluate("rps", [agents_py[i], agents_py[j]], configuration={"episodeSteps": 100}, num_episodes=1)
        scores[i][j] = result[0][0]  # сохраняем результат для агента i
        scores[j][i] = result[0][1]  # сохраняем результат для агента j

#Определяем победителя
vic = []
for i in range(n_agents):
    cnt_vic = sum(1 for j in range(n_agents) if scores[i][j] > 0)  # считаем победы агента
    vic.append(cnt_vic)

#Определяем индексы победителей
i_victors = [i for i, v in enumerate(vic) if v == max(vic)]
if len(i_victors) > 1:
    print("Победители турнира:", '\n', *[agents_py[i] for i in i_victors], "!")
else:
    print("Победитель турнира:", agents_py[i_victors[0]], "!")

Победители турнира: 
 pattern_agent.py q_learning_agent.py !


In [111]:
#Построение турнирной таблицы
df_scores = pd.DataFrame(scores, index=agents_py, columns=agents_py)
df_scores

,rock_agent.py,paper_agent.py,scissors_agent.py,random_agent.py,rock_paper_agent.py,paper_scissors_agent.py,mirror_agent.py,contradictious_agent.py,boring_agent.py,great_gambits_agent.py,gambits_shifter_agent.py,pattern_agent.py,q_learning_agent.py
rock_agent.py,0.0,-99.0,99.0,0.0,-51.0,0.0,0.0,0.0,-98.0,-25.0,-63.0,-94.0,-87.0
paper_agent.py,99.0,0.0,-99.0,0.0,48.0,-52.0,0.0,0.0,-98.0,0.0,-53.0,-93.0,-88.0
scissors_agent.py,-99.0,99.0,0.0,0.0,0.0,42.0,0.0,0.0,-96.0,0.0,-90.0,-95.0,-93.0
random_agent.py,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rock_paper_agent.py,51.0,-48.0,0.0,0.0,0.0,-22.0,0.0,0.0,0.0,0.0,-24.0,-20.0,-41.0
paper_scissors_agent.py,0.0,52.0,-42.0,0.0,22.0,0.0,0.0,0.0,-23.0,0.0,0.0,0.0,-41.0
mirror_agent.py,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-50.0,0.0,0.0,0.0,0.0
contradictious_agent.py,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,0.0,0.0,0.0,0.0
boring_agent.py,98.0,98.0,96.0,0.0,0.0,23.0,50.0,-46.0,0.0,0.0,-40.0,-50.0,38.0
great_gambits_agent.py,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.0,0.0,-22.0


****Вывод:****

Итого лучшие результаты показали pattern_agent (который анализирует последовательности из 3 ходов противника в поисках повторяющихся паттернов) и q_learning_agent (использующий Q-learning алгоритм с политикой Epsilon-Greedy для выбора хода). Это, в принципе, ожидаемо, потому что это два самых умных агента из всей моей "команды". 

In [127]:
evaluate(
    "rps", #environment to use - no need to change
    ["pattern_agent.py", "q_learning_agent.py"], #agents to evaluate
    configuration={"episodeSteps": 1000} #number of episodes 
)

[[-474.0, 474.0]]